In [1]:
import numpy as np
import pandas as pd 

In [2]:
player_df = pd.read_csv("candy-data.csv")

In [3]:
player_df.tail()

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
80,Twizzlers,0,1,0,0,0,0,0,0,0,0.220,0.116,45.466282
81,Warheads,0,1,0,0,0,0,1,0,0,0.093,0.116,39.011898
82,WelchÕs Fruit Snacks,0,1,0,0,0,0,0,0,1,0.313,0.313,44.375519
83,WertherÕs Original Caramel,0,0,1,0,0,0,1,0,0,0.186,0.267,41.904308
84,Whoppers,1,0,0,0,0,1,0,0,1,0.872,0.848,49.524113


In [4]:
numcols = ['chocolate','fruity','caramel','peanutyalmondy','nougat','crispedricewafer','hard','bar','pluribus','sugarpercent','pricepercent']
catcols = ['competitorname']

In [5]:
player_df = player_df[numcols+catcols]

In [6]:
player_df.head()

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,competitorname
0,1,0,1,0,0,1,0,1,0,0.732,0.860,100 Grand
1,1,0,0,0,1,0,0,1,0,0.604,0.511,3 Musketeers
2,0,0,0,0,0,0,0,0,0,0.011,0.116,One dime
3,0,0,0,0,0,0,0,0,0,0.011,0.511,One quarter
4,0,1,0,0,0,0,0,0,0,0.906,0.511,Air Heads


In [7]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [8]:
traindf = pd.DataFrame(traindf,columns=features)

In [9]:
y = traindf['pricepercent']>=0.500
X = traindf.copy()
del X['pricepercent']

In [10]:
X.shape

(85, 95)

In [11]:
y.tail()

80    False
81    False
82    False
83    False
84     True
Name: pricepercent, dtype: bool

In [12]:
len(X.columns)

95

In [13]:
num_feats=30

In [14]:
feature_name = list(X.columns)
feature_name

['chocolate',
 'fruity',
 'caramel',
 'peanutyalmondy',
 'nougat',
 'crispedricewafer',
 'hard',
 'bar',
 'pluribus',
 'sugarpercent',
 'competitorname_100 Grand',
 'competitorname_3 Musketeers',
 'competitorname_Air Heads',
 'competitorname_Almond Joy',
 'competitorname_Baby Ruth',
 'competitorname_Boston Baked Beans',
 'competitorname_Candy Corn',
 'competitorname_Caramel Apple Pops',
 'competitorname_Charleston Chew',
 'competitorname_Chewey Lemonhead Fruit Mix',
 'competitorname_Chiclets',
 'competitorname_Dots',
 'competitorname_Dum Dums',
 'competitorname_Fruit Chews',
 'competitorname_Fun Dip',
 'competitorname_Gobstopper',
 'competitorname_Haribo Gold Bears',
 'competitorname_Haribo Happy Cola',
 'competitorname_Haribo Sour Bears',
 'competitorname_Haribo Twin Snakes',
 'competitorname_HersheyÕs Kisses',
 'competitorname_HersheyÕs Krackel',
 'competitorname_HersheyÕs Milk Chocolate',
 'competitorname_HersheyÕs Special Dark',
 'competitorname_Jawbusters',
 'competitorname_Junior

In [15]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


In [16]:
cor_feature

['competitorname_ReeseÕs Peanut Butter cup',
 'competitorname_Junior Mints',
 'competitorname_Kit Kat',
 'competitorname_M&MÕs',
 'competitorname_Milk Duds',
 'competitorname_Milky Way',
 'competitorname_Mounds',
 'competitorname_Dots',
 'competitorname_Nestle Crunch',
 'competitorname_Nestle Smarties',
 'competitorname_Nik L Nip',
 'competitorname_Payday',
 'competitorname_Peanut M&Ms',
 'competitorname_Peanut butter M&MÕs',
 'competitorname_Pop Rocks',
 'competitorname_HersheyÕs Special Dark',
 'competitorname_ReeseÕs pieces',
 'competitorname_Jawbusters',
 'competitorname_Nestle Butterfinger',
 'competitorname_HersheyÕs Milk Chocolate',
 'caramel',
 'nougat',
 'pluribus',
 'sugarpercent',
 'hard',
 'crispedricewafer',
 'peanutyalmondy',
 'bar',
 'fruity',
 'chocolate']

In [17]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

30 selected features


In [18]:
chi_feature

['chocolate',
 'fruity',
 'caramel',
 'peanutyalmondy',
 'nougat',
 'crispedricewafer',
 'hard',
 'bar',
 'pluribus',
 'competitorname_Nestle Butterfinger',
 'competitorname_Nestle Crunch',
 'competitorname_Nestle Smarties',
 'competitorname_Nik L Nip',
 'competitorname_One quarter',
 'competitorname_Payday',
 'competitorname_Peanut M&Ms',
 'competitorname_Peanut butter M&MÕs',
 'competitorname_Pop Rocks',
 'competitorname_ReeseÕs Peanut Butter cup',
 'competitorname_ReeseÕs pieces',
 'competitorname_ReeseÕs stuffed with pieces',
 'competitorname_Ring pop',
 'competitorname_Rolo',
 'competitorname_Snickers',
 'competitorname_Snickers Crisper',
 'competitorname_Sugar Babies',
 'competitorname_Swedish Fish',
 'competitorname_Tootsie Roll Juniors',
 'competitorname_Twix',
 'competitorname_Whoppers']

In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 95 features.
Fitting estimator with 85 features.
Fitting estimator with 75 features.
Fitting estimator with 65 features.
Fitting estimator with 55 features.
Fitting estimator with 45 features.
Fitting estimator with 35 features.
30 selected features


In [20]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

30 selected features


In [21]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(X, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

22 selected features


In [22]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(X, y)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

30 selected features


In [23]:
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,peanutyalmondy,True,True,True,True,True,True,6
2,hard,True,True,True,True,True,True,6
3,fruity,True,True,True,True,True,True,6
4,crispedricewafer,True,True,True,True,True,True,6
5,chocolate,True,True,True,True,True,True,6
6,bar,True,True,True,True,True,True,6
7,sugarpercent,True,False,True,True,True,True,5
8,competitorname_Pop Rocks,True,True,True,True,True,False,5
9,competitorname_Nik L Nip,True,True,True,True,True,False,5
10,competitorname_Dots,True,False,True,True,True,True,5
